<a href="https://colab.research.google.com/github/yms0606/SecuLayer/blob/main/%EC%9E%90%EC%9C%A8%EC%A3%BC%ED%96%89%20%EC%84%BC%EC%84%9C%EC%9D%98%20%EC%95%88%ED%85%8C%EB%82%98%20%EC%84%B1%EB%8A%A5%20%EC%98%88%EC%B8%A1%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%EC%9E%90%EC%9C%A8%EC%A3%BC%ED%96%89_%EC%84%BC%EC%84%9C%EC%9D%98_%EC%95%88%ED%85%8C%EB%82%98_%EC%84%B1%EB%8A%A5_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install optuna

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
from lightgbm import LGBMRegressor
from tqdm import tqdm

import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor

In [3]:
train = pd.read_csv('/content/drive/MyDrive/ML study/LG/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML study/LG/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/ML study/LG/sample_submission.csv')
y_feature_spec_info = pd.read_csv('/content/drive/MyDrive/ML study/LG/meta/y_feature_spec_info.csv')

In [4]:
target_list = []
min_list = []
max_list = []

for i in range(len(y_feature_spec_info)):
  target_list.append(y_feature_spec_info['Feature'][i])
  min_list.append(y_feature_spec_info['최소'][i])
  max_list.append(y_feature_spec_info['최대'][i])

In [5]:
X = train[[
    'X_01', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_12', 'X_14', 'X_15', 'X_16', 'X_17', 'X_19', 'X_20',
    'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29', 'X_30',
    'X_31', 'X_32', 'X_33', 'X_35', 'X_37', 'X_38', 'X_39', 'X_40',
    'X_41', 'X_42', 'X_43', 'X_44', 'X_46', 'X_49', 'X_50',
    'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56'
    ]]
y = train[[
    'Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06', 'Y_07', 'Y_08', 'Y_09', 'Y_10',
    'Y_11', 'Y_12', 'Y_13', 'Y_14'
    ]]

In [6]:
target = test[X.columns]

In [7]:
def lg_nrmse(gt, preds):
  all_nrmse = []
  for idx in range(14):
    rmse = mean_squared_error(gt.iloc[:,idx],preds[:,idx], squared = False)
    nrmse = rmse / np.mean(np.abs(gt.iloc[:,idx]))
    all_nrmse.append(nrmse)
  score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
  return score

In [8]:
kf = KFold(n_splits= 10, random_state= 2022, shuffle = True)

In [14]:
def objective_cat(trial):
  x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=2022)

  params = {
        'tree method' : 'gpu_hist',
        'objective': 'regression',
        'verbose': -1,
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 2, 8, step=1, log=True),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),
        'reg_alpha': 0,
        'reg_lambda': 0,
        'max_depth': trial.suggest_int('max_depth',3, 8),
        'learning_rate': 0.1,
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, step=100),
        'min_child_samples': 20,
        'subsample': 1,
        'random_state': 42,
  }

  model = MultiOutputRegressor(LGBMRegressor(**params))
  model.fit(x_train, y_train)
  pred = model.predict(x_test)
  rmse = lg_nrmse(y_test, pred)

  return rmse

In [16]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_cat, n_trials=10)

[I 2023-06-28 06:23:48,654] A new study created in memory with name: no-name-e5806525-d389-4f19-9e30-ab5009a6cd6f
<ipython-input-14-f49a42a25458>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),
[I 2023-06-28 06:24:14,633] Trial 0 finished with value: 1.9385610440721406 and parameters: {'num_leaves': 4, 'colsample_bytree': 0.7462889158032554, 'max_depth': 4, 'n_estimators': 500}. Best is trial 0 with value: 1.9385610440721406.
<ipython-input-14-f49a42a25458>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),


[LightGBM] [Warning] Unknown parameter: method


[I 2023-06-28 06:24:41,039] Trial 1 finished with value: 1.949418191003636 and parameters: {'num_leaves': 2, 'colsample_bytree': 0.8565877965139893, 'max_depth': 5, 'n_estimators': 600}. Best is trial 0 with value: 1.9385610440721406.
<ipython-input-14-f49a42a25458>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),


[LightGBM] [Warning] Unknown parameter: method


[I 2023-06-28 06:25:03,962] Trial 2 finished with value: 1.9376220893553342 and parameters: {'num_leaves': 8, 'colsample_bytree': 0.8271222840082486, 'max_depth': 4, 'n_estimators': 300}. Best is trial 2 with value: 1.9376220893553342.
<ipython-input-14-f49a42a25458>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),


...

[I 2023-06-28 06:26:29,341] Trial 7 finished with value: 1.9375750462570127 and parameters: {'num_leaves': 5, 'colsample_bytree': 0.818335099831512, 'max_depth': 4, 'n_estimators': 500}. Best is trial 7 with value: 1.9375750462570127.
<ipython-input-14-f49a42a25458>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),


[LightGBM] [Warning] Unknown parameter: method


[I 2023-06-28 06:26:38,394] Trial 8 finished with value: 1.960045389495525 and parameters: {'num_leaves': 2, 'colsample_bytree': 0.8893546573408012, 'max_depth': 7, 'n_estimators': 100}. Best is trial 7 with value: 1.9375750462570127.
<ipython-input-14-f49a42a25458>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),


[LightGBM] [Warning] Unknown parameter: method


[I 2023-06-28 06:27:04,580] Trial 9 finished with value: 1.9494775687817618 and parameters: {'num_leaves': 2, 'colsample_bytree': 0.828568122645579, 'max_depth': 8, 'n_estimators': 600}. Best is trial 7 with value: 1.9375750462570127.


In [17]:
params = study.best_params
params['metric'] = 'rmse'
params['objective'] = 'regression'
params

{'num_leaves': 5,
 'colsample_bytree': 0.818335099831512,
 'max_depth': 4,
 'n_estimators': 500,
 'metric': 'rmse',
 'objective': 'regression'}

In [18]:
NRMSES = []
pred = np.zeros((target.shape[0],14))
for i, idx in enumerate(kf.split(X,y)):
  tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
  val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]

  model = MultiOutputRegressor(LGBMRegressor(**params))
  model.fit(tr_x, tr_y)

  val_pred = model.predict(val_x)
  NRMSE = lg_nrmse(val_y, val_pred)
  print(f'{i+1} Fold NRMSE = {NRMSE}')
  NRMSES.append(NRMSE)

  fold_pred = model.predict(target) / kf.n_splits
  pred += fold_pred

1 Fold NRMSE = 1.959348660327108
2 Fold NRMSE = 1.907472853041897
3 Fold NRMSE = 1.9735389693590055
4 Fold NRMSE = 1.9104200303791943
5 Fold NRMSE = 1.957411937153299
6 Fold NRMSE = 1.9732548827899807
7 Fold NRMSE = 1.9490766291689592
8 Fold NRMSE = 1.9891225461768607
9 Fold NRMSE = 1.9787689290778918
10 Fold NRMSE = 1.9793684560094398


In [19]:
np.mean(NRMSES)

submission.iloc[:,1:] = pred

submission.to_csv('submssion.csv', index=False)